In [2]:
# pip install -r requirements.txt

In [3]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

In [4]:
# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [6]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [7]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

/home/ita/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [9]:
np.shape(X_train)

(8000, 12)

In [10]:
# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [11]:
# Initialising the ANN
classifier = Sequential()

In [12]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 12))
# classifier.add(Dropout(rate = 0.1))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dropout(rate = 0.1))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
8000/8000 [==============================] - 1s 152us/step - loss: 0.4758 - accuracy: 0.7959
Epoch 2/10
8000/8000 [==============================] - 1s 127us/step - loss: 0.4294 - accuracy: 0.7960
Epoch 3/10
8000/8000 [==============================] - 1s 138us/step - loss: 0.4288 - accuracy: 0.7960
Epoch 4/10
8000/8000 [==============================] - 1s 123us/step - loss: 0.4286 - accuracy: 0.7960
Epoch 5/10
8000/8000 [==============================] - 1s 132us/step - loss: 0.4280 - accuracy: 0.7960
Epoch 6/10
8000/8000 [==============================] - 1s 126us/step - loss: 0.4240 - accuracy: 0.8076
Epoch 7/10
8000/8000 [==============================] - 1s 133us/step - loss: 0.4109 - accuracy: 0.8124
Epoch 8/10
8000/8000 [==============================] - 1s 133us/step - loss: 0.3958 - accuracy: 0.8180
Epoch 9/10
8000/8000 [==============================] - 1s 119us/step - l

In [13]:
# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [14]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000
Exited: Yes"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000, 1]])))
new_prediction = (new_prediction > 0.5)

In [15]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1497   98]
 [ 255  150]]


In [16]:
# Part 4 - Evaluating, Improving and Tuning the ANN

# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import Dropout
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [17]:
print(accuracies)
print(mean)
print(variance)

[0.82375002 0.79000002 0.81875002 0.82125002 0.84625    0.82499999
 0.82999998 0.82499999 0.81999999 0.83625001]
0.8236250042915344
0.013743741570548466


In [18]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [19]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

In [20]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [21]:
classifier = KerasClassifier(build_fn = build_classifier)

In [22]:
parameters = {'batch_size': [5, 10],
              'epochs': [5, 10],
              'optimizer': ['adam', 'rmsprop']}

In [23]:
grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [24]:
grid_search = grid_search.fit(X_train, y_train)

acy: 0.8265
Epoch 9/10
7200/7200 [==============================] - 2s 307us/step - loss: 0.4148 - accuracy: 0.8292
Epoch 10/10
7200/7200 [==============================] - 2s 337us/step - loss: 0.4216 - accuracy: 0.8231
Epoch 1/10
7200/7200 [==============================] - 3s 398us/step - loss: 0.4861 - accuracy: 0.7956
Epoch 2/10
7200/7200 [==============================] - 2s 297us/step - loss: 0.4273 - accuracy: 0.7956
Epoch 3/10
7200/7200 [==============================] - 3s 367us/step - loss: 0.4210 - accuracy: 0.7956
Epoch 4/10
7200/7200 [==============================] - 2s 325us/step - loss: 0.4145 - accuracy: 0.8129
Epoch 5/10
7200/7200 [==============================] - 2s 331us/step - loss: 0.4175 - accuracy: 0.8218
Epoch 6/10
7200/7200 [==============================] - 2s 318us/step - loss: 0.4112 - accuracy: 0.8235
Epoch 7/10
7200/7200 [==============================] - 2s 315us/step - loss: 0.4129 - accuracy: 0.8217
Epoch 8/10
7200/7200 [=============================

In [25]:
best_parameters = grid_search.best_params_

In [26]:
best_accuracy = grid_search.best_score_

In [27]:
print(best_parameters)

{'epochs': 10, 'optimizer': 'rmsprop', 'batch_size': 5}


In [28]:
print(best_accuracy)

0.830875
